# Path to images zipfile

In [ ]:
!tar -xvf  '/content/drive/MyDrive/Copy of nyu_depth_images.tar' -C './'

In [ ]:
!pip install -U keras
!pip install transformers
!pip install gputil
!pip install psutil
!pip install humanize
!pip install nltk

In [ ]:
import os, re, random, zipfile, pickle, torch, logging
import numpy as np, torch.nn as nn, pandas as pd,\
torch.nn.functional as F, matplotlib.pyplot as plt,\
seaborn as sn
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot, Tokenizer
from time import time, sleep
from models import SelfVQA
from string import punctuation
from torch.optim import SGD, Adam, lr_scheduler
from sklearn.metrics import confusion_matrix
from torch.utils.data import (
    TensorDataset,
    Dataset,
    DataLoader, 
    Subset, 
    RandomSampler, SequentialSampler
)
from sklearn.metrics import (
    roc_curve, 
    roc_auc_score, 
    precision_recall_fscore_support
)
from PIL import Image, ImageFilter, ImageDraw
from torchvision.models import vgg16_bn
from torchvision.transforms import ToTensor
from transformers import get_linear_schedule_with_warmup
# import nltk
# nltk.download('wordnet')
# from nltk.corpus import wordnet

In [ ]:
def download_dirs(dirlist):
    from google.colab import files
    for dirname in dirlist:
        for filename in os.listdir(dirname):
            filename = os.path.join(dirname, filename)
            files.download(filename)
            
def get_dataLoader(dataset, splits, batch_sizes, shuffle_indices, **kwargs):
    n_samples = len(dataset)
    indices = list(range(n_samples))
    if shuffle_indices:
        np.random.shuffle(indices)
    split_index = int(n_samples*splits[0])
    train_indices = indices[:split_index]
    valid_indices = indices[split_index:]
    train_dataset = Subset(dataset, train_indices)
    valid_dataset = Subset(dataset, valid_indices)
    train_loader = DataLoader(
        train_dataset, shuffle=True, batch_size=batch_sizes[0], **kwargs
    )
    valid_loader = DataLoader(
        valid_dataset, shuffle=False, batch_size=batch_sizes[1], **kwargs
    )
    return train_loader, valid_loader

def fix_seed(seed_value, random_lib=False, numpy_lib=False, torch_lib=False):
    if random_lib:
        random.seed(seed_value)
    if numpy_lib:
        np.random.seed(seed_value)
    if torch_lib:
        torch.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)

def setup_logger(name, format=None, level=logging.DEBUG, handlers=None, log_file='default.log'):
    logging.basicConfig(
        level=level, 
        format=format if format else '%(asctime)s [%(levelname)s] %(message)s',
        handlers=handlers if handlers else [
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger(name)
    logger.setLevel(level)

def cal_accuracy(pred_labels, true_labels):
    _, pred_labels = pred_labels.max(dim=1)
    true_labels = true_labels.view(-1)
    return torch.sum(pred_labels == true_labels).item() / true_labels.size()[0]

def cal_wups(pred_labels, true_labels, label2word):
    print(pred_labels)
    print(true_labels)
    pred_labels = pred_labels.tolist()
    true_labels = true_labels.tolist()
    wups_scores = []
    for i in range(len(pred_labels)):
        pred_syn = wordnet.synsets(label2word[pred_labels[i]])
        true_syn = wordnet.synsets(label2word[true_labels[i]])
        mid_score = []
        for psyn in pred_syn:
            for tsyn in true_syn:
                score = psyn.wup_similarity(tsyn)
                if score:
                    mid_score.append(score)
        mean_score = np.mean(mid_score)
        if not np.isnan(mean_score):
            wups_scores.append(mean_score)
    return np.mean(wups_scores)

In [ ]:
class SelfDataset(Dataset):

    def __init__(self, ant_df, images_dir, tokenizer=None, transform=ToTensor()):
        self.ant_df = ant_df
        self.encoded_matrix, self.vocab_size, self.tokenizer = encode_txt(
            list(ant_df.question.values), tokenizer
        )
        self.encoded_matrix = torch.as_tensor(self.encoded_matrix)
        self.image_names = list(ant_df.image_name.values)
        self.images_dir = images_dir
        self.labels = torch.as_tensor(ant_df.answer_id.values)
        self.transform = transform

    def __getitem__(self, item):
        question = self.encoded_matrix[item]
        image_name = os.path.join(self.images_dir, self.image_names[item])
        img = Image.open(image_name)
        if self.transform:
            img = self.transform(img)
        label = self.labels[item]
        return question, img, label

    def __len__(self):
        return len(self.labels)

class TrainTest():
    def __init__(self, model, optimizer, scheduler, criterion, logger=None):
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.criterion = criterion
        self.logger = logger
        self.tr_metrics = {
            'train_loss':[],
            'train_accuracy':[],
            'valid_loss':[],
            'valid_accuracy':[],
        }
        
    def train(
        self, train_loader, valid_loader, 
        num_epochs, device, eval_interval,
        clip=None, model_path=None, save_per_epoch=None,
        results_path=None, defaults=None, **kwargs
    ):
        total_itrs = num_epochs*len(train_loader)
        num_tr, total_tr_loss, itr = 0, 0, 0
        self.model.train()
        for epoch in range(num_epochs):
            for i, (questions, imgs, labels) in enumerate(train_loader):
                questions = questions.to(device)
                imgs = imgs.to(device)
                labels = labels.to(device)
                self.optimizer.zero_grad()
                output = self.model(
                    questions,
                    imgs
                )
                tr_loss = self.criterion(output, labels.view(-1))
                # nn.utils.clip_grad_norm_(model.parameters(), clip)
                if self.logger:
                    self.logger.info(f'Training: {itr}/{total_itrs} -- loss: {tr_loss.item()}')
                tr_loss.backward()
                self.optimizer.step()
                self.scheduler.step()
                num_tr += 1
                total_tr_loss += tr_loss
                if itr % eval_interval == 0 or itr+1 == total_itrs:
                    self.tr_metrics['train_loss'].append(total_tr_loss.cpu().item()/num_tr)
                    tr_accuracy = cal_accuracy(output, labels)
                    self.tr_metrics['train_accuracy'].append(tr_accuracy)
                    num_tr, total_tr_loss = 0, 0
                    val_loss = 0
                    self.model.eval()
                    val_accuracy = []
                    with torch.no_grad():
                        for i, (val_questions, val_imgs, val_labels) in enumerate(valid_loader):
                            val_questions = val_questions.to(device)
                            val_imgs = val_imgs.to(device)
                            val_labels = val_labels.to(device)
                            self.optimizer.zero_grad()
                            val_output = self.model(
                                val_questions,
                                val_imgs
                            )
                            val_loss += self.criterion(val_output, val_labels.view(-1))
                            val_accuracy.append(cal_accuracy(val_output, val_labels))
                    self.tr_metrics['valid_accuracy'].append(np.mean(val_accuracy))
                    self.tr_metrics['valid_loss'].append(val_loss.cpu().item()/len(valid_loader))
                    self.model.train()
                    if self.logger:
                        self.logger.info(f'Training: iteration: {itr}/{total_itrs} -- epoch: {epoch} -- '
                        f' train_loss: {self.tr_metrics["train_loss"][-1]:.3f} -- train_accuracy: {self.tr_metrics["train_accuracy"][-1]:.2f}'
                        f' valid_loss: {self.tr_metrics["valid_loss"][-1]:.3f} -- valid_accuracy: {self.tr_metrics["valid_accuracy"][-1]:.2f}')
                itr += 1
            if model_path and results_path and ((epoch+1) % save_per_epoch == 0) and epoch != 0:
                self.save_model(epoch+1, model_path, f'{epoch+1}_epochs_train')
                self.save_results(results_path, f'{epoch+1}_epochs_train', self.tr_metrics)
        if model_path and results_path:
            self.save_model(epoch+1, model_path, f'{epoch+1}_epochs_last_train')
            self.save_results(results_path, f'{epoch+1}_epochs_last_train', self.tr_metrics)
            
    def save_model(self, epoch, model_path, name):
        model_dir = '/'.join(model_path.split('/')[:-1])
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        torch.save(
            {
                'epoch': epoch,
                'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'scheduler_state_dict': self.scheduler.state_dict(),
                'loss': self.tr_metrics['valid_loss'][-1],
            }, os.path.join(model_dir, f'model_{name}.pt')
        )
        if self.logger:
            self.logger.info(f'Training: model saved to: {model_dir}/model_{name}.pt')
    
    def save_results(self, results_path, name, results):
        results_dir = '/'.join(results_path.split('/')[:-1])
        if not os.path.exists(results_dir):
            os.makedirs(results_dir)
        with open(os.path.join(results_dir, f'results_{name}.pkl'), 'wb') as save_file:
            pickle.dump(results, save_file)
        if self.logger:
            self.logger.info(f'Training: results saved to: {results_dir}/resutls_{name}.pkl')

    def test(self, test_loader, device, label2word, results_path=None, defaults=None):
        test_accuracy, test_true, test_pred = [], [], []
        test_loss = 0
        self.model.eval()
        with torch.no_grad():
            for i, (test_questions, test_imgs, test_labels) in enumerate(test_loader):
                test_questions = test_questions.to(device)
                test_imgs = test_imgs.to(device)
                test_labels = test_labels.to(device)
                self.optimizer.zero_grad()
                test_output = self.model(
                    test_questions,
                    test_imgs
                )
                test_loss += self.criterion(test_output, test_labels.view(-1))
                test_accuracy.append(cal_accuracy(test_output, test_labels))
                test_true.append(test_labels.cpu())
                test_pred.append(test_output.cpu().max(dim=1)[1])
        test_true = torch.cat(test_true)
        test_pred = torch.cat(test_pred)
        test_loss = test_loss.cpu().item()/len(test_loader)
        test_accuracy = np.mean(test_accuracy)
        test_wups = cal_wups(test_pred, test_true, label2word)
        prf = precision_recall_fscore_support(
            test_true,
            test_pred,
            labels=list(label2word.keys()),
            average='weighted'
        )
        confm = confusion_matrix(test_true, test_pred, labels=list(label2word.keys()))
        self.ts_metrics = {
            'loss':test_loss,
            'accuracy':test_accuracy,
            'wups':test_wups,
            'precision':prf[0],
            'recall':prf[1],
            'f1_score':prf[2],
            'confusion_matrix':confm
        }
        if self.logger:
            print(f'Testing: test_loss: {test_loss:.3f} -- test_accurcy: {test_accuracy:.2f} -- test_wups: {test_wups:.2f}')
        if results_path:
            self.save_results(results_path, f'test', self.ts_metrics)

In [ ]:
def resize_images(resize_shape, images_path, img_dist_path='rescaled_images'):
    if not os.path.exists(img_dist_path):
        os.makedirs(img_dist_path)
    else: return img_dist_path
    image_names = sorted(os.listdir(images_path))
    img_counter = 0
    for img_file in image_names:
        img = Image.open(os.path.join(images_path, img_file))
        img.thumbnail(resize_shape, Image.ANTIALIAS)
        new_path = os.path.join(img_dist_path, img_file)
        img.save(new_path, format='PNG')
        img_counter += 1
    return img_dist_path

def create_df(file_path, answer_mapping=None):
    with open(file_path, 'r') as annotation_file:
        questions = []
        answers = []
        image_names = []
        chars = set('_,')
        lines = annotation_file.readlines()
        for i in range(0, len(lines)-1, 2):
            question = lines[i].strip().split()
            answer = lines[i+1].strip()
            if answer_mapping and answer not in answer_mapping.keys():
                continue
            if not any((c in chars) for c in answer):
                image_name = question.pop(-2)
                image_names.append(image_name[image_name.index('image'):]+'.png')
                answers.append(answer)
                questions.append(' '.join(question[:-3])+'?')
    df = pd.DataFrame(
        list(zip(image_names, questions, answers)), columns=['image_name', 'question', 'answer']
    ).sort_values(['image_name']).reset_index(drop=True)
    valid_answers = np.sort(df.answer.unique())
    if not answer_mapping:
        answer_mapping = dict(zip(valid_answers, range(len(valid_answers))))
    df['answer_id'] = df.answer.apply(lambda x: answer_mapping[x])
    return df, answer_mapping

def encode_txt(txt_list, tokenizer=None):
    if tokenizer is None:
        print('train tokenizer')
        tokenizer = Tokenizer(oov_token='OOV')
        tokenizer.fit_on_texts(txt_list)
    vocab_size = len(tokenizer.word_index) + 1
    encoded_txt = tokenizer.texts_to_sequences(txt_list)
    encoded_matrix = pad_sequences(
        encoded_txt,
        padding='pre',
        truncating='pre'
    )
    return encoded_matrix, vocab_size, tokenizer

In [ ]:
cuda_flag = True if torch.cuda.is_available() else False
lrlast = .001
lrmain = .00001
n_iters = 10000
num_epochs = 12
eval_interval = 150
save_model = True
device = torch.device('cuda' if cuda_flag else 'cpu')
params = {'num_workers': 2, 'pin_memory': True} if cuda_flag else {}
data_splits = [0.9, 0.1]
batch_sizes = [8, 8]
seed = 20214
fix_seed(seed, random_lib=True, numpy_lib=True, torch_lib=True)

# Set Path to images directory and annotation files for training and test data

In [ ]:
resize_shape = (200, 200)
resized_dir = resize_images(resize_shape, './nyu_depth_images/', './rescaled_images/')
tr_ant_path = './qa.894.raw.train.txt'
train_df, answer_mapping = create_df(tr_ant_path)
label2word = {v: k for k, v in answer_mapping.items()}
train_dataset = SelfDataset(train_df, resized_dir)
ts_ant_path = './qa.894.raw.test.txt'
test_df, _ = create_df(ts_ant_path, answer_mapping)
test_dataset = SelfDataset(test_df, resized_dir, train_dataset.tokenizer)

train_loader, valid_loader = get_dataLoader(
    train_dataset,
    data_splits,
    batch_sizes,
    shuffle_indices=True, **params
)
test_loader = DataLoader(
    test_dataset, shuffle=False, batch_size=8, **params
)

# Set path to GloVe 300d embedding fil

In [ ]:
embedded_words = dict()
glove_path = '/content/drive/MyDrive/Copy of glove.42B.300d.txt'
glove_file = open(glove_path)
for line in glove_file:
    values = line.split(' ')
    word = values[0]
    words_weights = np.array(values[1:], dtype='float32')
    embedded_words[word] = words_weights
glove_file.close()

In [ ]:
mean_embedding = np.array(list(embedded_words.values())).mean(axis=0)
embedding_dim = len(embedded_words[next(iter(embedded_words))])
embedding_matrix = np.zeros((train_dataset.vocab_size, embedding_dim))
for word, index in train_dataset.tokenizer.word_index.items():
    embedding_vector = embedded_words.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
    else:
        embedding_matrix[index] = mean_embedding

del embedded_words

In [ ]:
output_size = len(answer_mapping)
hidden_dim = 150
n_layers = 1
lstm_drop = 0.5

model = SelfVQA(
    train_dataset.vocab_size,
    output_size,
    embedding_dim,
    hidden_dim,
    n_layers,
    lstm_drop,
    bidirectional=True
).to(device)

embedding_matrix = torch.as_tensor(embedding_matrix, dtype=torch.float)
model.embedding.weight.data.copy_(embedding_matrix)
model.embedding.weight.requires_grad=False
for param in model.vgg.parameters():
    param.requires_grad = False

In [ ]:
optimizer = Adam(model.parameters(), lr=0.001)
# optimizer = SGD(model.parameters(), lr=lrmain)
# scheduler = lr_scheduler.StepLR(optimizer, step_size=1)
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps = 0,
    num_training_steps = num_epochs*len(train_loader),
)
open('metrics.log', 'w').close()
logger = setup_logger(name='track_logger', level=logging.INFO, log_file='metrics.log')

In [ ]:
traintest = TrainTest(model, optimizer, scheduler, F.nll_loss, logger)
traintest.train(
    train_loader,
    valid_loader,
    num_epochs,
    device,
    eval_interval,
    model_path='./models/',
    save_per_epoch=10,
    results_path='./results/',
    clip=5
)

In [ ]:
traintest.test(test_loader, device, label2word, results_path='./results/')

In [ ]:
download_dirs(['./results/'])

In [ ]:
traintest.tr_metrics

In [ ]:
answer_mapping

In [ ]:
def report_results(
    results_dir, train_results_file, eval_interval, just_train=False,
    test_results_file=None, class_mapping=None, **kwargs
):
    with open(os.path.join(results_dir, train_results_file), 'rb') as results_file:
        train_results = pickle.load(results_file)
    iterations = [i*eval_interval for i in range(len(train_results['train_loss']))]
    plt.plot(iterations, train_results['train_loss'], label=f'train loss')
    plt.plot(iterations, train_results['valid_loss'], label=f'valid loss')
    plt.title('Loss')
    plt.xlabel('iterations')
    plt.ylabel('loss')
    plt.legend()
    plt.savefig(os.path.join(results_dir, 'train-validation-loss.png'), dpi=300, bbox_inches="tight")
    plt.figure()
    plt.plot(iterations, train_results['train_accuracy'], label=f'train accuracy')
    plt.plot(iterations, train_results['valid_accuracy'], label=f'valid accuracy')
    plt.title('Accuracy')
    plt.xlabel('iterations')
    plt.ylabel('accuracy')
    plt.legend()
    plt.savefig(os.path.join(results_dir, 'train-validation-accuracy.png'), dpi=300, bbox_inches="tight")
    if not just_train:
        with open(os.path.join(results_dir, test_results_file), 'rb') as results_file:
            test_results = pickle.load(results_file)
        class_mapping = sorted(
            list(class_mapping.items()), key=lambda x: x[1]
        )
        classes = [x[0] for x in class_mapping]
        plt.figure()
        test_confm = pd.DataFrame(test_results['confusion_matrix'], classes, classes)
        sn.set(font_scale=1)
        sn.heatmap(
            test_confm, 
            annot=kwargs['conf_annot'] if 'conf_annot' in kwargs else True, 
            annot_kws={"size": kwargs['font-size'] if 'font-size' in kwargs else 10}
        )
        plt.autoscale(True)
        plt.savefig(os.path.join(results_dir, 'test-confusion-matrix.png'), dpi=300, bbox_inches="tight")
        print(f'{"*"*20} Test Metrics: {"*"*20}\n'
              f'Loss: {test_results["loss"]:.3f}\n'
              f'Accuracy: {test_results["accuracy"]:.3f}\n'
              f'WUPS: {test_results["wups"]:.3f}\n'
              f'Weighted Precision: {test_results["precision"]:.3f}\n'
              f'Weighted Recall: {test_results["recall"]:.3f}\n'
              f'Weighted F1-score: {test_results["f1_score"]:.3f}\n'
              f'{"*"*55}')

In [ ]:
results_dir = 'VQA-results'
test_results_file = 'results_test.pkl'
train_results_file = 'results_12_epochs_last_train.pkl'
report_results(
    results_dir, train_results_file, eval_interval,
    test_results_file=test_results_file,
    class_mapping=dict(zip(range(len(answer_mapping)), range(len(answer_mapping)))), conf_annot=False
)